## Change all file paths if you want to train yourself

In [1]:
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import os

# model expects the tarining files will be made available in the `DATASET_BASE_PATH` 
# where there will be separate folders for files in each class 
DATASET_BASE_PATH = "traffic_dataset"

In [48]:
 
audio_files = []  # List of audio file paths
labels = []  # List of corresponding labels

folders = [f for f in os.listdir(DATASET_BASE_PATH) if '.' not in f]
for folder in folders:
    files = os.listdir(f"{DATASET_BASE_PATH}/{folder}")
    files = [f"{DATASET_BASE_PATH}/{folder}/{file}" for file in files if file.endswith(".wav")]
    
    labels += [str(folder)] * len(files)
    audio_files += files

In [49]:

print("***** Audio *****")
display(audio_files)
print("##### Labels #####")
# display(labels)

***** Audio *****


['traffic_dataset/idle/8380230f-a447-4952-ad0e-79c7ca0a959b.wav',
 'traffic_dataset/idle/5eb6180f-53bf-4986-a7b9-19a94b097b61.wav',
 'traffic_dataset/idle/74ae54f5-1fb9-4da2-a9ca-99bc4cc73b0b.wav',
 'traffic_dataset/idle/cb26ac00-617e-48f4-ab54-5ec0ff027235.wav',
 'traffic_dataset/idle/fd7f1150-399a-47e4-9d23-2ca7bcd61abd.wav',
 'traffic_dataset/idle/02eeb989-09c0-4005-89d9-3199c608e18e.wav',
 'traffic_dataset/idle/38b2423c-4828-4e46-8939-e2e3e3f152a7.wav',
 'traffic_dataset/idle/2f5c4698-2c6c-484c-97ed-36cff10ead96.wav',
 'traffic_dataset/idle/a30cc371-3702-4574-881d-2eef3bde4871.wav',
 'traffic_dataset/idle/413ec05e-edd9-4422-8ef0-9e9987e9845e.wav',
 'traffic_dataset/idle/e74a7ec4-dbcb-41f8-8c72-2037944ca9b1.wav',
 'traffic_dataset/idle/85c118c8-f135-4ac7-b31b-d633588d415c.wav',
 'traffic_dataset/idle/840546af-a0f8-40f4-b8fa-1137d50ecb26.wav',
 'traffic_dataset/idle/66c4a06b-ad65-474c-8ed9-64d09bcd645a.wav',
 'traffic_dataset/idle/0b75b13b-f58c-4e51-8837-a922ef239396.wav',
 'traffic_

##### Labels #####


In [50]:
def extract_features(y, sr):
    
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    rmse = librosa.feature.rms(y=y)[0]
    
    features = np.hstack((np.mean(mfccs, axis=1), 
                          np.mean(spectral_centroid), 
                          np.mean(spectral_rolloff), 
                          np.mean(zcr), 
                          np.mean(rmse)))
    return features


In [51]:

X = [extract_features(*librosa.load(audio_file, sr=None)) for audio_file in audio_files if audio_file.endswith(".wav")]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]


In [58]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=39,)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=39)

In [59]:

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 89.13%


In [60]:
accuracy_rand = {}
for i in range(0, 101):
    model = RandomForestClassifier(n_estimators=100, random_state=i,)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_rand[i] = accuracy

print(max(accuracy_rand, key=lambda x: accuracy_rand[x]))
    

39


In [61]:
print(max(accuracy_rand, key=lambda x: accuracy_rand[x]))
print(accuracy_rand[39])

39
0.8913043478260869


# Testing 

In [63]:
file_path = "../full_audio_files/test/light, main gate  22 Mar, 10.12 AM.wav"

# extract features of input file to predict 
test_data = []
data, sr = librosa.load(file_path)
for i in range(0, len(data), (sr*5)):
    test_data.append(extract_features(data[i:(i+(sr*5))], sr))                      

# prediction
preds = model.predict(test_data)
aggregate_class_index = round(np.mean(preds))
forecast = label_encoder.inverse_transform([aggregate_class_index])[0]
forecast


'medium'

## Saving the Model 

In [50]:
# import pickle

# with open('model.pkl', 'wb') as file:
#     pickle.dump(model, file)

# with open('label_encoder.pkl', 'wb') as file:
#     pickle.dump(label_encoder, file)